# Proyecto 02 - Data Science - Web Scraping de Apartamentos en Alquiler en Zona 14 - Gian Paolo Robelo 20230117

Este proyecto utiliza **BeautifulSoup** y **requests** para realizar scraping de información de apartamentos en alquiler en la página de **Mapainmueble**.

## Descripción del Proyecto

Se extrae la siguiente información de las propiedades listadas en **https://mapainmueble.com/apartamentos-en-alquiler-zona-14/** para **7 páginas**:

- **Título**
- **Precio**
- **Dormitorios**
- **Baños**
- **Tamaño**
- **Propietario**
- **Descripción**
- **Fotos**
- **Enlace**


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# URL de la página web
BASE_URL = "https://mapainmueble.com/apartamentos-en-alquiler-zona-14/page/{}"
headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' 'AppleWebKit/537.36 (KHTML, like Gecko) ''Chrome/115.0.0.0 Safari/537.36')
}

In [3]:
apartments = []

# Recorrer las 7 páginas
for page in range(1, 8):
    url = BASE_URL.format(page)
    response = requests.get(url, headers=headers, verify=False)
    
    if response.status_code != 200:
        print(f"Error al acceder a la página {page} (status code: {response.status_code})")
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Se buscan los contenedores de las propiedades usando la clase "listing_wrapper"
    listings = soup.find_all("div", class_="listing_wrapper")
    
    if not listings:
        print(f"No se encontraron listados en la página {page}. Verifica el selector.")
    
    for listing in listings:
        try:
            # Extraer título y enlace desde la etiqueta <h4> y su <a> interna
            title_elem = listing.find("h4")
            if title_elem:
                link_elem = title_elem.find("a")
                title = link_elem.text.strip() if link_elem else "N/A"
                link = link_elem.get("href", "N/A") if link_elem else "N/A"
            else:
                title = "N/A"
                link = "N/A"
            
            # Extraer precio desde el div de precio
            price_elem = listing.find("div", class_="listing_unit_price_wrapper")
            price = price_elem.text.strip() if price_elem else "N/A"
            
            # Extraer detalles: dormitorios, baños y tamaño
            bedrooms_elem = listing.find("span", class_="inforoom")
            bedrooms = bedrooms_elem.text.strip() if bedrooms_elem else "N/A"
            
            bathrooms_elem = listing.find("span", class_="infobath")
            bathrooms = bathrooms_elem.text.strip() if bathrooms_elem else "N/A"
            
            size_elem = listing.find("span", class_="infosize")
            size = size_elem.text.strip() if size_elem else "N/A"
            
            # Extraer el nombre del propietario desde la sección del agente
            owner_elem = listing.find("div", class_="property_agent_wrapper")
            if owner_elem:
                owner_link = owner_elem.find("a")
                owner = owner_link.text.strip() if owner_link else "N/A"
            else:
                owner = "N/A"
            
            # Extraer la descripción
            desc_elem = listing.find("div", class_="listing_details the_grid_view")
            description = desc_elem.text.strip() if desc_elem else "N/A"
            
            # Extraer el número de fotos
            photos_elem = listing.find("div", class_="property_media")
            num_photos = "N/A"
            if photos_elem:
                photos_text = photos_elem.text.strip()
                num_photos = ''.join(filter(str.isdigit, photos_text)) if any(c.isdigit() for c in photos_text) else "N/A"
            
            apartments.append({
                "Título": title,
                "Precio": price,
                "Dormitorios": bedrooms,
                "Baños": bathrooms,
                "Tamaño": size,
                "Propietario": owner,
                "Descripción": description,
                "Fotos": num_photos,
                "Enlace": link
            })
        except Exception as e:
            print(f"Error procesando un anuncio: {e}")


C:\Users\gianp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapainmueble.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\gianp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapainmueble.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\gianp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: Insecur

In [4]:
# Convertir la lista de propiedades en un DataFrame y guardarlo en CSV
df = pd.DataFrame(apartments)
df.to_csv("apartamentos_zona_14.csv", index=False, encoding="utf-8-sig")

print("Scraping completado. Datos guardados en 'apartamentos_zona_14.csv'")

Scraping completado. Datos guardados en 'apartamentos_zona_14.csv'
